In [ ]:
import requests
import pandas as pd
import os
import time
from datetime import datetime
from tqdm import tqdm

# --- Konfiguration ---
# URL-Endpunkt für den Datenexport
EXPORT_URL = "https://transparency.entsoe.eu/balancing/r3/cbmpsForAfrrStandardProduct/export"

# Der Ordner, in dem die heruntergeladenen Dateien gespeichert werden
OUTPUT_DIR = "entsoe_afrr_data"

# Der gewünschte Zeitraum. Hier: 4 Jahre inklusive 2024
START_DATE = "2021-01-01"
END_DATE = "2024-12-31"

# HTTP-Header, um einen echten Browser zu simulieren und Probleme zu vermeiden
HEADERS = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'Origin': 'https://transparency.entsoe.eu',
    'Referer': 'https://transparency.entsoe.eu/balancing/r3/cbmpsForAfrrStandardProduct/show'
}

def download_data_for_day(session, date_to_fetch):
    """
    Diese Funktion formuliert die Anfrage für einen einzelnen Tag, sendet sie
    und speichert die resultierende Excel-Datei.
    """
    # Das Datum in das für die Anfrage benötigte Format bringen (TT.MM.JJJJ)
    date_str_payload = date_to_fetch.strftime('%d.%m.%Y')
    
    # Die Nutzdaten (Payload) für die POST-Anfrage.
    # Diese entsprechen den Formularfeldern, die der Browser senden würde.
    # Wichtig: viewType ist 'TABLE', da die Tabellenansicht exportiert wird.
    payload = [
        ('name', ''),
        ('defaultValue', 'false'),
        ('viewType', 'TABLE'),
        ('areaType', 'COMBINED_LFA_SCA_IPA'),
        ('atch', 'false'),
        ('dateTime.dateTime', f'{date_str_payload} 00:00|CET|DAYTIMERANGE'),
        ('dateTime.endDateTime', f'{date_str_payload} 00:00|CET|DAYTIMERANGE'),
        ('direction.values', 'Up'),
        ('direction.values', 'Down'),
        ('A16A96', 'A16A96'),
        ('A01', 'A01'),
        ('marketArea.values', 'CTY|10Y1001A1001A83F!LFA|10YDE-EON------1'),
        ('dateTime.timezone', 'CET_CEST'),
        ('dateTime.timezone_input', 'CET (UTC+1) / CEST (UTC+2)'),
        ('exportType', 'xlsx')
    ]

    try:
        # Senden der POST-Anfrage mit den definierten Daten und Headern
        response = session.post(EXPORT_URL, data=payload, headers=HEADERS, timeout=30)
        
        # Prüft, ob die Anfrage erfolgreich war (Status-Code 200-299)
        response.raise_for_status()

        # Sicherstellen, dass die Antwort eine Excel-Datei ist und kein HTML-Fehler
        if 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' in response.headers.get('Content-Type', ''):
            # Dateinamen für die Speicherung generieren (z.B. afrr_data_2022-01-15.xlsx)
            file_date_str = date_to_fetch.strftime('%Y-%m-%d')
            filename = os.path.join(OUTPUT_DIR, f"afrr_standard_product_{file_date_str}.xlsx")

            # Die Datei im Zielordner speichern
            with open(filename, 'wb') as f:
                f.write(response.content)
            # Erfolgsmeldung wird durch die tqdm-Fortschrittsanzeige ersetzt
        else:
            # Manchmal schickt der Server eine HTML-Seite mit einer Fehlermeldung statt der Datei
            print(f"WARNUNG: Download für {date_str_payload} fehlgeschlagen. Server lieferte unerwarteten Inhaltstyp: {response.headers.get('Content-Type')}")

    except requests.exceptions.RequestException as e:
        print(f"FEHLER beim Download für {date_str_payload}: {e}")

def main():
    """
    Die Hauptfunktion, die den gesamten Prozess steuert.
    """
    # 1. Erstellt den Ausgabeordner, falls er nicht existiert
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"Ordner '{OUTPUT_DIR}' wurde erstellt.")

    # 2. Erstellt eine Liste aller Tage im gewünschten Zeitraum
    dates = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
    
    print(f"Starte Download für {len(dates)} Tage von {START_DATE} bis {END_DATE}...")

    # 3. Eine 'Session' wird verwendet, um Verbindungen und Cookies wiederzuverwenden (effizienter)
    with requests.Session() as session:
        # Die Schleife (Loop), die über jeden Tag iteriert
        # tqdm sorgt für eine Fortschrittsanzeige
        for date in tqdm(dates, desc="Lade Daten herunter"):
            download_data_for_day(session, date)
            
            # Eine kurze Pause von 1 Sekunde zwischen den Anfragen.
            # Das ist wichtig, um den Server nicht zu überlasten und eine IP-Sperre zu vermeiden.
            time.sleep(1)
            
    print("\nDownload abgeschlossen.")
    print(f"Alle Dateien wurden im Ordner '{OUTPUT_DIR}' gespeichert.")


# Standard-Python-Konvention: Führt die main()-Funktion aus, wenn das Skript direkt gestartet wird.
if __name__ == "__main__":
    main()